# 5m-Exploratory Data Analysis

In [1]:
# !pip install pandas

In [2]:
import pandas as pd

## Load data

In [3]:
calendar = pd.read_csv("./kaggle-5m-data/calendar.csv") 
sell_prices = pd.read_csv("./kaggle-5m-data/sell_prices.csv") 
sales = pd.read_csv("./kaggle-5m-data/sales_train_validation.csv")

## Describe and prepare data
Go through each dataset and prepare them for building pipelines

### calendar data 
Contains information about the dates on which the products are sold.

In [4]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [5]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [6]:
# check the columns
cols_to_check = ["year", "weekday", "month", "event_name_1", "event_name_2",
                 "event_type_1", "event_type_2", "snap_CA", "snap_TX", "snap_WI"]

for col in cols_to_check:
    print(f"\n--- {col} ---")
    print(calendar[col].value_counts(dropna=False))  # including NaN


--- year ---
year
2012    366
2013    365
2014    365
2015    365
2011    337
2016    171
Name: count, dtype: int64

--- weekday ---
weekday
Saturday     282
Sunday       282
Monday       281
Tuesday      281
Wednesday    281
Thursday     281
Friday       281
Name: count, dtype: int64

--- month ---
month
3     186
5     186
4     180
2     170
6     169
1     158
7     155
8     155
10    155
12    155
9     150
11    150
Name: count, dtype: int64

--- event_name_1 ---
event_name_1
NaN                    1807
Pesach End                6
Ramadan starts            6
SuperBowl                 6
NBAFinalsEnd              6
NBAFinalsStart            6
MemorialDay               6
Mother's day              6
Purim End                 6
StPatricksDay             6
LentWeek2                 6
LentStart                 6
PresidentsDay             6
ValentinesDay             6
MartinLutherKingDay       5
OrthodoxChristmas         5
EidAlAdha                 5
NewYear                   5
Chanuka

In [7]:
# date format conversion: object --> date-time
calendar['date'] = pd.to_datetime(calendar['date'])

In [8]:
# missing values: event_name_1, event_name_2, event_type_1, and event_type_2 fields may have missing values. 
# missing values can be replaced with 'No_event'
calendar.fillna({'event_name_1': 'No_event', 'event_name_2': 'No_event',
                 'event_type_1': 'No_event', 'event_type_2': 'No_event'}, inplace=True)

In [9]:
# convert d to int
calendar['d'] = calendar['d'].str[2:].astype(int)

In [10]:
# delete redundant columns: weekday, month, year. We can calculate them based on the date.
calendar.drop(["weekday", "wday", "month", "year"], axis = 1)

,date,wm_yr_wk,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,1,No_event,No_event,No_event,No_event,0,0,0
1,2011-01-30,11101,2,No_event,No_event,No_event,No_event,0,0,0
2,2011-01-31,11101,3,No_event,No_event,No_event,No_event,0,0,0
3,2011-02-01,11101,4,No_event,No_event,No_event,No_event,1,1,0
4,2011-02-02,11101,5,No_event,No_event,No_event,No_event,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,1965,No_event,No_event,No_event,No_event,0,1,1
1965,2016-06-16,11620,1966,No_event,No_event,No_event,No_event,0,0,0
1966,2016-06-17,11620,1967,No_event,No_event,No_event,No_event,0,0,0
1967,2016-06-18,11621,1968,No_event,No_event,No_event,No_event,0,0,0


In [11]:
# Keep only calendar rows where 'd' <= 1913 to align with sales data
calendar = calendar[calendar['d'] <= 1913]

calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1908,2016-04-20,11612,Wednesday,5,4,2016,1909,No_event,No_event,No_event,No_event,0,0,0
1909,2016-04-21,11612,Thursday,6,4,2016,1910,No_event,No_event,No_event,No_event,0,0,0
1910,2016-04-22,11612,Friday,7,4,2016,1911,No_event,No_event,No_event,No_event,0,0,0
1911,2016-04-23,11613,Saturday,1,4,2016,1912,No_event,No_event,No_event,No_event,0,0,0
1912,2016-04-24,11613,Sunday,2,4,2016,1913,No_event,No_event,No_event,No_event,0,0,0


### sell_prices data
Contains information about the price of the products sold per store and date.

In [12]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [13]:
sell_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [14]:
# check if data has any missing values
sell_prices.isnull().sum()

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

In [15]:
# check that the price is zero or negative: 
(sell_prices["sell_price"] <= 0).sum()  # 检查非正价格

0

### sales data
Contains the historical daily unit sales data per product and store [d_1 - d_1913]

In [16]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [17]:
# check some columns
cols_to_check = ["state_id", "store_id", "cat_id", "dept_id", "item_id"]

for col in cols_to_check:
    print(f"\n--- {col} ---")
    print(sales[col].value_counts(dropna=False))  # including NaN


--- state_id ---
state_id
CA    12196
TX     9147
WI     9147
Name: count, dtype: int64

--- store_id ---
store_id
CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    3049
TX_2    3049
TX_3    3049
WI_1    3049
WI_2    3049
WI_3    3049
Name: count, dtype: int64

--- cat_id ---
cat_id
FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: count, dtype: int64

--- dept_id ---
dept_id
FOODS_3        8230
HOUSEHOLD_1    5320
HOUSEHOLD_2    5150
HOBBIES_1      4160
FOODS_2        3980
FOODS_1        2160
HOBBIES_2      1490
Name: count, dtype: int64

--- item_id ---
item_id
HOBBIES_1_001      10
FOODS_2_210        10
FOODS_2_201        10
FOODS_2_202        10
FOODS_2_203        10
                   ..
HOUSEHOLD_1_464    10
HOUSEHOLD_1_465    10
HOUSEHOLD_1_466    10
HOUSEHOLD_1_467    10
FOODS_3_827        10
Name: count, Length: 3049, dtype: int64


In [18]:
# remove the redundant column ID as it can be derived from other fields
# reshape the data
date_cols = [col for col in sales.columns if col.startswith('d_')]
sales_long = sales.melt(
    id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    value_vars=date_cols,
    var_name='day',
    value_name='sales'
)

sales_long

,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...
58327365,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1
58327366,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327367,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327368,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3


In [19]:
# convert d to int
sales_long['day'] = sales_long['day'].str[2:].astype(int)

In [20]:
# check some columns
cols_to_check = ["state_id", "store_id", "cat_id", "dept_id", "item_id", "day", "sales"]

for col in cols_to_check:
    print(f"\n--- {col} ---")
    print(sales_long[col].value_counts(dropna=False))  # including NaN


--- state_id ---
state_id
CA    23330948
TX    17498211
WI    17498211
Name: count, dtype: int64

--- store_id ---
store_id
CA_1    5832737
CA_2    5832737
CA_3    5832737
CA_4    5832737
TX_1    5832737
TX_2    5832737
TX_3    5832737
WI_1    5832737
WI_2    5832737
WI_3    5832737
Name: count, dtype: int64

--- cat_id ---
cat_id
FOODS        27489810
HOUSEHOLD    20029110
HOBBIES      10808450
Name: count, dtype: int64

--- dept_id ---
dept_id
FOODS_3        15743990
HOUSEHOLD_1    10177160
HOUSEHOLD_2     9851950
HOBBIES_1       7958080
FOODS_2         7613740
FOODS_1         4132080
HOBBIES_2       2850370
Name: count, dtype: int64

--- item_id ---
item_id
HOBBIES_1_001      19130
FOODS_2_210        19130
FOODS_2_201        19130
FOODS_2_202        19130
FOODS_2_203        19130
                   ...  
HOUSEHOLD_1_464    19130
HOUSEHOLD_1_465    19130
HOUSEHOLD_1_466    19130
HOUSEHOLD_1_467    19130
FOODS_3_827        19130
Name: count, Length: 3049, dtype: int64

--- day ---
da

## Merge data
use chunked merging for the large data

In [21]:
#sales_merged = sales_long.merge(calendar, left_on='day', right_on='d', how='left')

In [22]:
import gc  # Garbage collector

# randomly extract 0.001% of the data (to avoid memory problems)
sales_sample = sales_long.sample(frac=0.00001, random_state=42)

# merge sampled data
sales_merged = sales_sample.merge(
    calendar,
    left_on='day',
    right_on='d',
    how='left'
)

gc.collect()  # Force garbage collection

sales_merged.head()

,item_id,dept_id,cat_id,store_id,state_id,day,sales,date,wm_yr_wk,weekday,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,FOODS_3_548,FOODS_3,FOODS,WI_2,WI,1560,0,2015-05-07,11514,Thursday,...,5,2015,1560,No_event,No_event,No_event,No_event,1,1,0
1,FOODS_3_231,FOODS_3,FOODS,WI_1,WI,892,6,2013-07-08,11324,Monday,...,7,2013,892,No_event,No_event,No_event,No_event,1,0,1
2,FOODS_3_319,FOODS_3,FOODS,CA_3,CA,1096,31,2014-01-28,11353,Tuesday,...,1,2014,1096,No_event,No_event,No_event,No_event,0,0,0
3,HOUSEHOLD_1_098,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,1364,0,2014-10-23,11438,Thursday,...,10,2014,1364,No_event,No_event,No_event,No_event,0,0,0
4,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_3,CA,1658,0,2015-08-13,11528,Thursday,...,8,2015,1658,No_event,No_event,No_event,No_event,0,1,0


In [23]:
# merge the merged data with sell_prices
final_data = sales_merged.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

final_data.head()

,item_id,dept_id,cat_id,store_id,state_id,day,sales,date,wm_yr_wk,weekday,...,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_3_548,FOODS_3,FOODS,WI_2,WI,1560,0,2015-05-07,11514,Thursday,...,2015,1560,No_event,No_event,No_event,No_event,1,1,0,2.58
1,FOODS_3_231,FOODS_3,FOODS,WI_1,WI,892,6,2013-07-08,11324,Monday,...,2013,892,No_event,No_event,No_event,No_event,1,0,1,4.98
2,FOODS_3_319,FOODS_3,FOODS,CA_3,CA,1096,31,2014-01-28,11353,Tuesday,...,2014,1096,No_event,No_event,No_event,No_event,0,0,0,1.00
3,HOUSEHOLD_1_098,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,1364,0,2014-10-23,11438,Thursday,...,2014,1364,No_event,No_event,No_event,No_event,0,0,0,7.98
4,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_3,CA,1658,0,2015-08-13,11528,Thursday,...,2015,1658,No_event,No_event,No_event,No_event,0,1,0,0.48
